# 智谱API测试

## 使用 ChatZhipuAI

In [3]:
!pip install -U langchain_zhipu==4.0.20
# !pip install pydantic==1.10.13
# !pip install pydantic==2.6.2

  Attempting uninstall: langchain_zhipu
    Found existing installation: langchain_zhipu 4.0.18
    Uninstalling langchain_zhipu-4.0.18:
      Successfully uninstalled langchain_zhipu-4.0.18


In [1]:
from dotenv import load_dotenv, find_dotenv
load_dotenv(find_dotenv(), override=True)
import os
os.chdir('..')

In [2]:
from langchain_zhipu import ChatZhipuAI
from langchain_core.prompts import ChatPromptTemplate

In [3]:
llm = ChatZhipuAI()
llm.get_num_tokens(text="abc")

1

In [7]:
llm.get_token_ids(text="hi")

[6151]

In [4]:
llm.invoke("hi")

AIMessage(content='你好👋！我是人工智能助手智谱清言（ChatGLM），很高兴见到你，欢迎问我任何问题。')

In [ ]:
await llm.ainvoke("hi")

AIMessage(content='你好👋！我是人工智能助手智谱清言（ChatGLM），很高兴见到你，欢迎问我任何问题。')

In [5]:
for chunk in llm.stream("hi"):
  print(chunk.content, end="|", flush=True)

Hello| |👋!| I|'|m| Chat|GL|M|（|智|谱|清|言|）,| the| artificial| intelligence| assistant|,| nice| to| meet| you|.| Feel| free| to| ask| me| any| questions|.||

In [8]:
async for chunk in llm.astream("hi"):
  print(chunk.content, end="|", flush=True)

Hello|!| How| can| I| assist| you| today|?||

In [10]:
async for chunk in llm.astream_events("hi", version="v1"):
  print(chunk)

{'event': 'on_chat_model_start', 'run_id': '4f202b64-1034-455b-9bc4-5fc32e027565', 'name': 'ChatZhipuAI', 'tags': [], 'metadata': {}, 'data': {'input': 'hi'}}
{'event': 'on_chat_model_stream', 'run_id': '4f202b64-1034-455b-9bc4-5fc32e027565', 'tags': [], 'metadata': {}, 'name': 'ChatZhipuAI', 'data': {'chunk': AIMessageChunk(content='你好')}}
{'event': 'on_chat_model_stream', 'run_id': '4f202b64-1034-455b-9bc4-5fc32e027565', 'tags': [], 'metadata': {}, 'name': 'ChatZhipuAI', 'data': {'chunk': AIMessageChunk(content='👋！')}}
{'event': 'on_chat_model_stream', 'run_id': '4f202b64-1034-455b-9bc4-5fc32e027565', 'tags': [], 'metadata': {}, 'name': 'ChatZhipuAI', 'data': {'chunk': AIMessageChunk(content='我是')}}
{'event': 'on_chat_model_stream', 'run_id': '4f202b64-1034-455b-9bc4-5fc32e027565', 'tags': [], 'metadata': {}, 'name': 'ChatZhipuAI', 'data': {'chunk': AIMessageChunk(content='人工智能')}}
{'event': 'on_chat_model_stream', 'run_id': '4f202b64-1034-455b-9bc4-5fc32e027565', 'tags': [], 'metada

In [6]:
tools = [
    {
        "type": "function",
        "function": {
            "name": "query_train_info",
            "description": "根据用户提供的信息，查询对应的车次",
            "parameters": {
                "type": "object",
                "properties": {
                    "departure": {
                        "type": "string",
                        "description": "出发城市或车站",
                    },
                    "destination": {
                        "type": "string",
                        "description": "目的地城市或车站",
                    },
                    "date": {
                        "type": "string",
                        "description": "要查询的车次日期",
                    },
                },
                "required": ["departure", "destination", "date"],
            },
        }
    }
]

In [7]:
llm = ChatZhipuAI().bind(tools=tools)

In [9]:
llm

ChatZhipuAI(client=<langchain_zhipu._client.ZhipuAI object at 0x11936d750>)

In [8]:
llm.invoke("给我讲个一句话笑话")

AIMessage(content='一句话笑话：为什么大象不用电脑？因为它们怕鼠标。')

In [9]:
llm.invoke("你能帮我查询2024年1月1日从北京南站到上海的火车票吗？")

AIMessage(content='', additional_kwargs={'tool_calls': [{'id': 'call_8513525371557843149', 'function': {'arguments': '{"date":"2024-01-01","departure":"北京南站","destination":"上海"}', 'name': 'query_train_info'}, 'type': 'function'}]})

In [10]:
for chunk in llm.stream("你能帮我查询2024年1月1日从北京南站到上海的火车票吗？"):
  print(chunk)

content='' additional_kwargs={'tool_calls': [{'index': 0, 'id': 'call_8513531693749783917', 'function': {'arguments': '{"date":"2024-01-01","departure":"北京南站","destination":"上海"}', 'name': 'query_train_info'}, 'type': 'function'}]}
content=''


In [11]:
llm = ChatZhipuAI()
llm.invoke("贾玲最新的作品是什么？")

AIMessage(content='贾玲最新的作品是电影《热辣滚烫》。这部影片在2024年春节档期上映，讲述了女主角励志减肥，成为女拳击手的故事。在这部电影中，贾玲本人也进行了大幅度的减重，瘦身100斤，引起了广泛的关注和讨论。')

In [12]:
tools = [
    {
        "type": "web_search",
        "web_search": {
            "search_query": "贾玲演过的相声"
        }
    }
]
llm = ChatZhipuAI().bind(tools=tools)
llm.invoke("列举一个贾玲的作品")

AIMessage(content='贾玲的代表作品之一是《大话捧逗》。这个作品是她与搭档白凯南共同表演的一段相声，通过幽默诙谐的方式，讽刺和揭示了社会上的某些现象和人物，给观众带来了许多欢笑。该作品在2010年中央电视台春节联欢晚会上演出，并且获得了“我最喜爱的春晚节目”曲艺组三等奖。')

In [13]:
tools = [
    {
        "type": "web_search",
        "web_search": {
            "search_query": "贾玲演过的电影"
        }
    }
]
llm = ChatZhipuAI().bind(tools=tools)
llm.invoke("从背景信息中找出贾玲的作品")

AIMessage(content='根据提供的参考信息，以下是贾玲的作品列表：\n\n1. 电视剧作品：\n   - 《月里青山淡如画》（饰演罗母，2022年）\n   - 《幸福还会来敲门》（饰演张嫂，2020年）\n   - 《欢喜猎人》（饰演贾小玲，2020年）\n   - 《爆笑先森》（饰演KTV老板，2016年）\n   - 《百变五侠之我是大明星》（饰演宫煮，2016年）\n\n2. 电影作品：\n   - 《你好，李焕英》（自导自演，2023年）\n   - 《悠然见南山》\n   - 《幸福马上来》\n   - 《祖宗十九代》\n   - 《摆渡人》\n   - 《万万没想到》\n   - 《我和我的家乡之回乡之路》（主演之一）\n   - 《热辣滚烫》（导演及主演，2024年）\n\n以上是贾玲在影视方面的一些主要作品，包括她自导自演的电影《你好，李焕英》和导演的第二部电影《热辣滚烫》。此外，她还有在多部电视剧中的演出经历。')